In [244]:
# !pip3 install langchain_community

In [245]:
from langchain_community.llms import Ollama

### Read transaction data

In [246]:
# Read the transactions from csv
import pandas as pd
df = pd.read_csv("2024-05-22_transaction_download.csv")
df.head()

,Transaction Date,Posted Date,Card No.,Description,Category,Debit,Credit
0,2024-05-21,2024-05-21,9770,"CSC ServiceWorks, Inc.",Other Services,10.00,NaN
1,2024-05-19,2024-05-21,9770,24 HOUR FITNESS - 513 - M,Other,6.54,NaN
2,2024-05-19,2024-05-20,9770,SQ *RESENDIZ FARMS,Merchandise,27.86,NaN
3,2024-05-19,2024-05-20,9770,SQ *CLARAVALE FARM COMPAN,Merchandise,11.00,NaN
4,2024-05-19,2024-05-20,9770,SQ *MOUNTAIN VIEW SUN,Merchandise,28.00,NaN


In [247]:
#Get unique transactions in the Name / Description column
unique_transactions = df['Description'].unique()
len(unique_transactions)

160

In [248]:
response = llm.invoke("Can you add an appropriate category to the following expenses. For example: SQ *RESENDIZ FARMS  - Groceries, CSC ServiceWorks, Inc. - Household Expenses, etc.. Categories should be less than 4 words. " + ",".join(unique_transactions[1:50]))
response = response.split("\n")

In [249]:
response

["I've added categories to the expenses. Here is the updated list:",
 '',
 '**Groceries**',
 '',
 '* SQ *RESENDIZ FARMS',
 '* JL PRODUCE',
 '* ARCO#06157SAN RAFAEL SMO',
 '* SP POMPONIO RANCH',
 '* TST* MILLER & LUX - PROVI',
 '* TST* SWEET MAPLE - PALO A',
 '',
 '**Fitness**',
 '',
 '* 24 HOUR FITNESS',
 '',
 '**Household Expenses**',
 '',
 '* CSC ServiceWorks, Inc.',
 '',
 '**Food Delivery**',
 '',
 '* UBER EATS',
 '',
 '**Subscription Services**',
 '',
 '* APPLE.COM/BILL',
 '* CHATGPT SUBSCRIPTION',
 '* PAYPAL *UBER BV',
 '* PAYPAL *LAGOSEC INC',
 '* PAYPAL *UBER EATS',
 '',
 '**Insurance**',
 '',
 '* LEMONADE INSURANCE',
 '',
 '**Transportation**',
 '',
 '* UBER',
 '* ARCO#06157SAN RAFAEL SMO',
 '',
 '**Restaurants**',
 '',
 "* MCDONALD'S",
 '* FAMILYMART',
 '* SHANA THAI',
 '',
 '**Events**',
 '',
 '* OAKLAND GREEK FESTIVA',
 '',
 '**Gardening**',
 '',
 '* PP*LIVE EARTH FARM LLC',
 '* SQ *RAMOS FARMS LLC',
 '* SQ *C FARM',
 '* SQ *TWIN GIRLS FARMS',
 '* SQ *APRICOTKING ORCHARDS',


### Categorise bank transactions with Llama3

In [250]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start,stop,step):
  for i in range(start,stop,step):
    yield i
  yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 30, 60, 90, 120, 150, 160]

In [251]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: SQ *RESENDIZ FARMS  - Groceries, CSC ServiceWorks, Inc. - Household Expenses, Aegean - Travel etc.. Categories should be less than 4 words, description and category seperated by a dash. Some places might have japanese names if category can't be found they can be specified as Travel, especially if it's during March 26 and April 9." + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})

    # New code to handle strings without ' - '
    def split_and_fill(s):
        parts = s.split(' - ', maxsplit=1)
        if len(parts) == 1:
            parts.append('NaT')  # Default category
        return pd.Series(parts)

    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].apply(split_and_fill)

    return categories_df

In [252]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['Here are the categorized expenses:', '', '* ISS Catering Services De Meern - Food', '* SQ *RESENDIZ FARMS - Groceries', '* CSC ServiceWorks, Inc. - Household Expenses', '* Aegean - Travel', '* Taxi Utrecht - Transportation', '* Etos AMSTERDAM NLD - Shopping', '* Bistro Bar Amsterdam - Dining', '', 'Let me know if you need any further assistance!']


,Transaction vs category,Transaction,Category
0,Here are the categorized expenses:,Here are the categorized expenses:,Unknown Category
1,,,Unknown Category
2,* ISS Catering Services De Meern - Food,* ISS Catering Services De Meern,Food
3,* SQ *RESENDIZ FARMS - Groceries,* SQ *RESENDIZ FARMS,Groceries
4,"* CSC ServiceWorks, Inc. - Household Expenses","* CSC ServiceWorks, Inc.",Household Expenses
5,* Aegean - Travel,* Aegean,Travel
6,* Taxi Utrecht - Transportation,* Taxi Utrecht,Transportation
7,* Etos AMSTERDAM NLD - Shopping,* Etos AMSTERDAM NLD,Shopping
8,* Bistro Bar Amsterdam - Dining,* Bistro Bar Amsterdam,Dining
9,,,Unknown Category


In [253]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Here is the list with categories added:', '', '* CSC ServiceWorks, Inc. - Household Expenses', '* 24 HOUR FITNESS - Gym Membership', '* SQ *RESENDIZ FARMS - Groceries', '* SQ *CLARAVALE FARM COMPAN - Travel (assuming travel-related expense)', '* SQ *MOUNTAIN VIEW SUN - Travel (assuming travel-related expense)', '* SQ *ADORABLE FRENCH BAKER - Food Delivery', '* SQ *OAKLAND GREEK FESTIVA - Travel (assuming travel-related expense)', '* CAPITAL ONE MOBILE PYMT - Bills Payable', '* APPLE.COM/BILL - Bills Payable', '* AEGEAN_WEB_ - Travel', '* COTFLT - Travel (assuming travel-related expense)', '* SQ *ROADSIDE ROTISSERIE L - Food Delivery', '* SP POMPONIO RANCH - Travel (assuming travel-related expense)', '* APPLE STORE #R002 - Electronics', '* TST* SWEET MAPLE - PALO A - Food Delivery', '* SQ *VERVE COFFEE ROASTERS - Coffee Shop', '* CARNICERIA LA SORPRESA - Grocery Store', '* VALERO 7528 - Gasoline', '* Shana Thai - Restaurant', '* EB GENAI SUMMIT SAN F - Travel (assuming travel-related 

In [254]:
categories_df_all


,Transaction vs category,Transaction,Category
0,Here is the list with categories added:,Here is the list with categories added:,Unknown Category
1,,,Unknown Category
2,"* CSC ServiceWorks, Inc. - Household Expenses","* CSC ServiceWorks, Inc.",Household Expenses
3,* 24 HOUR FITNESS - Gym Membership,* 24 HOUR FITNESS,Gym Membership
4,* SQ *RESENDIZ FARMS - Groceries,* SQ *RESENDIZ FARMS,Groceries
...,...,...,...
168,9. SQ *PETCO PARK - Entertainment (assuming it...,9. SQ *PETCO PARK,Entertainment (assuming it's a theme park or s...
169,10. PINKBERRY SAN DIEGO - Food & Drink,10. PINKBERRY SAN DIEGO,Food & Drink
170,11. KETTNER & G - Clothing,11. KETTNER & G,Clothing
171,12. TST* FALSE IDOL - Entertainment (assuming ...,12. TST* FALSE IDOL,Entertainment (assuming it's an event or concert)


In [255]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [256]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Unknown Category', 'Household Expenses', 'Gym Membership',
       'Groceries', 'Travel (assuming travel-related expense)',
       'Food Delivery', 'Bills Payable', 'Travel', 'Electronics',
       'PALO A - Food Delivery', 'Coffee Shop', 'Grocery Store',
       'Gasoline', 'Restaurant', 'Entertainment', 'Insurance',
       'Miscellaneous', 'Dining Out',
       'Transportation (assuming this is a gas station expense)',
       'Recreation/Sports', 'Online Services', 'Provisions',
       'Unknown (assuming this is a travel expense)', 'Farm Expenses',
       'Transportation', 'Rent/Mortgage',
       'Transportation (assuming this is a ride-sharing expense)',
       'Convenience Store', 'Accommodation', 'Food & Beverage', 'Fuel',
       'Banking', 'Personal Expenses', 'Dining', 'Retail',
       "Travel (since it's an international trip)", 'Unknown/Other',
       'Candy/Gift Shop', 'Gas Station', 'Gardening/Horticulture',
       'Gift Shop', 'Pet Supplies', 'Food', 'MISSION ST - Food'

In [257]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [258]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all.to_csv("categories_df_all.csv", index=False)

In [259]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
# df = pd.read_csv("2024-05-22_transaction_download.csv")
# df.loc[df['Transaction Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
# df = pd.merge(df, categories_df_all, left_on='Transaction Description', right_on='Transaction Description', how='left')
# df